In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import torch


In [2]:
llm = Ollama(model="mistral",  callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),num_gpu=1, base_url="http://localhost:11434")

In [3]:
modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/sid/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = DirectoryLoader("./data", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)


91

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)

461

In [6]:
# import torch
# torch.cuda.empty_cache()
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
torch.cuda.empty_cache()
gc.collect()

1276

In [8]:
from langchain.vectorstores import Chroma
persist_directory = './db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()

In [9]:
import torch
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
dev

'cuda:0'

In [10]:
retriever = vectordb.as_retriever(search_kwargs={'k': 7})

In [11]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k =  5

In [12]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                                       weights=[0.3, 0.7])

In [12]:
ensemble_retriever.get_relevant_documents("scholars admitted in 2019, 17")

[Document(page_content='4. Choose the option ID  \n\uf0b7 For scholars admitted in 2019 , the default option ID is 13  and for the \nscholars admitted in 2020 , the default option ID is 17 . \n\uf0b7 For the scholars admitted in 2018 or before , please select the \nappropriate option ID from t he list. (To know the available \noptions ,  Option Details - Click Here  ) \n5. Select appropriate Course Type  for each course.  \n \nCourse Type   Description  \nResearch Methodology  Research Methodology course', metadata={'page': 1, 'source': 'data/0.pdf'}),
 Document(page_content='scholar shall be the first author with the Guide as the corresponding author. If there is \nan approved Co -Guide, he/she can be a co -author in the publications  \n8.    There shall not be more than two  Guides for a research scholar.  \n  \nResearch Regulations applicable to the scholars who were admitted from 2015 to \n2018  \nPh.D. Regulations 2015 and 2017 shall continue and the following amendment to \nbe in

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=ensemble_retriever,
                                  return_source_documents=True)
     

In [15]:
def process_llm_response(query):
    llm_response = qa_chain(query)
    return llm_response['result']
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [21]:
query = "What do I need for Semester Abroad?"
process_llm_response(query)
# qa_chain(query)

 To participate in VIT's Semester Abroad Program, you need to be an undergraduate or final year student from any stream. You should register for additional 'honours' program elective courses listed in your curriculum to earn the 'Honours' credential. Students are not forced to take courses other than those prescribed under the 'honours' list as pre-requisite courses. You will carry out your Capstone Project in the campus of the University Abroad under the guidance of a Faculty member from that University and a guide from VIT. The progress of the work is monitored by both guides, and the final viva voce examination for the project will be conducted in VIT after successful completion.

You will need to follow the procedure given for paper seeing and reevaluation, if required, with payment of ₹100 for paper seeing and ₹300 for reevaluation. If your reevaluation mark is lesser than your actual mark, the best of the two marks will be considered.

You may contact different departments or aut

" To participate in VIT's Semester Abroad Program, you need to be an undergraduate or final year student from any stream. You should register for additional 'honours' program elective courses listed in your curriculum to earn the 'Honours' credential. Students are not forced to take courses other than those prescribed under the 'honours' list as pre-requisite courses. You will carry out your Capstone Project in the campus of the University Abroad under the guidance of a Faculty member from that University and a guide from VIT. The progress of the work is monitored by both guides, and the final viva voce examination for the project will be conducted in VIT after successful completion.\n\nYou will need to follow the procedure given for paper seeing and reevaluation, if required, with payment of ₹100 for paper seeing and ₹300 for reevaluation. If your reevaluation mark is lesser than your actual mark, the best of the two marks will be considered.\n\nYou may contact different departments o

In [ ]:
def main():
    st.title("VIT QA System")
    user_input = st.text_area("Enter Query")

    if st.button("Get response"):
        result = process_llm_response(user_input)
        st.write("Response:")
        # make text bigger
        st.write(f"**{result}**")
main()